# SPECFEM Users Workshop -- Day 1 (Oct. 5, 2022)

## Part 1A: Intro to SPECFEM2D

This notebook is meant to walk Users through an introduction to `SPECFEM2D`, from a) navigating a SPECFEM2D working directory, to b) generating a mesh, to c) running forward simulations. We make note of important files and steps to take when running forward simulations in SPECFEM. Background material is included to understand the task at hand. Understanding `SPECFEM2D` will help us draw parallels with `SPECFEM3D` in future notebooks. We will also use the information learned here to get a better understanding of how software like `SeisFlows` automate SPECFEM2D.

-----------

### Relevant Information

>__NOTE:__ These instructions should be run from inside the Docker container, using Jupyter Lab. The Docker container should have the adjTomo toolkit installed (SeisFlows, Pyatoa, PySEP), as well as SPECFEM2D and SPECEFM3D compiled with MPI. 

**Relevant Links:** 
- Day 1 Slides !!! ADD THIS !!!
- [Today's Notebook](https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/day_1a_intro_specfem.ipynb)
- Completed Notebook  !!! ADD THIS !!!

**Jupyter Quick Tips:**

- **Run cells** one-by-one by hitting the $\blacktriangleright$ button at the top, or by hitting `Shift + Enter`
- **Run all cells** by hitting the $\blacktriangleright\blacktriangleright$ button at the top, or by running `Run -> Run All Cells`
- **Currently running cells** that are still processing will have a `[*]` symbol next to them
- **Finished cells** will have a `[1]` symbol next to them. The number inside the brackets represents what order this cell has been run in.
- Commands that start with `!` are Bash commands (i.e., commands you would run from the terminal)
- Commands that start with `%` are Jupyter Magic commands.
- To time a task, put a `%time` before the command (e.g., `%time ! ls`)


## 1) Background !!! TODO !!!


Potential topics: 
- Seismic waveforms
- Numerical modeling
- Spectral element method
- Meshes


## 2) SPECFEM2D Tour

>__NOTE__: The `devel` branch of all SPECFEM versions contains the most up to date codebase

- GitHub Repository: https://github.com/geodynamics/specfem2d/tree/devel
- SPECFEM2D Manual: https://specfem2d.readthedocs.io/en/latest/

In [ ]:
%cd /home/scoped/specfem2d

### a) Binary Executables in *bin/* directory

In this workshop container, we have already downloaded (git clone), configured (choosing compilers and compiler options) and compiled (make all) SPECFEM2D. The binary executable files are located in the `bin/` directory. Each of these executables performs a different function in the package.

In [ ]:
# Let's have a look at the executables
! ls bin

The two most important executables we will be using today are `xmeshfem2D` and `xspecfem2D`. 
- `xmeshfem2D` is used to generate our numerical mesh, the skeleton of the domain upon which we run our numerical simulations. 
- `xspecfem2D` runs the spectral element solver, generating synthetic seismograms for a given source and set of stations.

Some other important executables we will use:
- `xsmooth_sem` smooths models, gradients and kernels by convolving them with a 2D Gaussian. Users can define the horizontal and vertical half-widths of the Gaussian.

### b) Metadata in *DATA/* directory

Data that the User will provide to SPECFEM should be stored in the *DATA/* directory. The most important files that we will concern ourselves with are the `Par_file`, `SOURCE` and `STATIONS` files.

-`Par_file`: The parameter file which allows the User to adjust parameters for a given simulation  
-`SOURCE`: Defines source characteristics (e.g., moment tensor, force). **NOTE**: SPECFEM2D and SPECFEM3D have a number of different types of available source files (e.g., SOURCE, FORCESOLUTION, CMTSOLUTION)  
-`STATIONS`: Defines station codes and locations (either Cartesian or geographic). **NOTE**: Station information may also be defined in the `Par_file`

The following commands open these files for the SPECFEM2D example problem

In [ ]:
! ls DATA

In [ ]:
! head -38 DATA/Par_file

In [ ]:
! cat DATA/SOURCE

In [ ]:
# In this example, the Par_file defines station information directly
! head -194 DATA/Par_file | tail -n 16

In [ ]:
# Other examples will define station information using STATIONS files
! head -5 EXAMPLES/Tape2007/DATA/STATIONS_checker

### c) Results stored in *OUTPUT_FILES/* directory

Any outputs generated by SPECFEM will be stored in the `OUTPUT_FILES/` directory. These include log and error messages, synthetic seismograms, figures, and output files of one executable that may be required by another.

>__NOTE__: SPECFEM also maintains a DATABASE directory (typically called `DATABASES_MPI/`) which is used to store large database files containing the entire GLL mesh and model, as well as intermediate files such as the saved forward wavefield. This directory may be the same as `OUTPUT_FILES/`, or may be it's own separate directory.

In [ ]:
# Currently empty because we have not run any executables
! ls OUTPUT_FILES

## 3) Running the mesher `xmeshfem2D` 

The first thing we need to do when approaching numerical simulations is to generate our numerical mesh. There are multiple approaches to meshing, such as using external software such as Trellis. During this workshop we will use SPECFEM's internal meshing software, known as `Meshfem`.

### a) Velocity Model Parameters

In SPECFEM2D, mesh parameters are defined in the `Par_file`. There are various parameter options we can use to customize our mesher run. The following parameter set allows us to read input values from the `Par_file`

`NPROC`: defines the number of processors used to partition the mesh  
`MODEL`=default:  

>__NOTE:__ In SPECFEM3D, Meshfem files are defined separately to provide more control over a 3D domain. These files are typically stored in `specfem3d/DATA/meshfem3D_files`. We will look at these later in the notebook.

In [ ]:
! head -273 DATA/Par_file | tail -n 34

In the output above we can see that our `Par_file` defines 4 separate material parameters, each with varying values for density and velocity. 

### b) Internal Mesher Parameters

The internal mesher also has another set of parameters that allows Users to provide interfaces, geometry, absorbing boundary conditions. These parameters are also used to distribute the material properties defined above, to specific regions of the mesh


In [ ]:
! head -320 DATA/Par_file | tail -n 29

### c) Setting Parameters

We will set a few `Par_file` parameters to tell SPECFEM to output a few additional files that will facilitate understanding the outputs of `xmeshfem2D`.

>__NOTE:__ We will use the `seisflows sempar` commmand to print and edit values from the SPECFEM2D `Par_file`. This is simply a convenience function but can be replaced by bash commands like 'cat' + 'awk', or by opening the `Par_file` with a text editor.

In [ ]:
# seisflows sempar -P ${Par_file} {key} {value:optional}
! seisflows sempar -P DATA/Par_file model
! seisflows sempar -P DATA/Par_file setup_with_binary_database 1
! seisflows sempar -P DATA/Par_file save_model binary

-------------
Explanations of the changes we are made include:

`MODEL`: Must be 'default' to use the model defined in the `Par_file` (default option)  
`setup_with_binary_database`: Writes database files in binary format   
`SAVE_MODEL`: Write model files in binary format  

### d) Run Meshfem Executable

Becuse this example problem is already set up for us, we can simply run the executable. `xmeshfem2D` will look for relevant data in the *DATA/* directory, and generate our numerical mesh, and model, in the *OUTPUT_FILES/* directory. 

>__NOTE__: WE will use MPI to run the executable, but set `-n=1` to specify that this is a serial task. We also redirect the output to a log file so that we can take a look at different parts of it.

In [ ]:
# The Par_file NPROC parameter must match the `-n` flag for MPI
! seisflows sempar -P DATA/Par_file nproc

In [ ]:
! mpirun -n 1 bin/xmeshfem2D > OUTPUT_FILES/output_meshfem2d.txt

In [ ]:
! ls OUTPUT_FILES

In [ ]:
# We can see that `xmeshfem2D` has created a STATIONS file for us
! ls DATA/

In [ ]:
! cat DATA/STATIONS

!!! Look at the output log file here !!!!

### 4) Running the solver `xspecfem2D`

The Solver `xspecfem2D` will now take the Database files generated by `xmeshfem2D` and run a forward simulation using the provided `SOURCE` and `STATIONS` files. The following set of parameters controls how the output synthetic seismograms are generated

In [ ]:
! head -168 DATA/Par_file | tail -n 32

Some of the more important parameters explained are:

`seismotype`: Sets the units of the output seismograms. This example outputs in units of 'displacement'  
`USER_T0`: Defines when the earlist starting time is, prior to time step 0. This allows some zero padding before initiating the source, and is useful e.g., in cases where you have very short source-receiver distances  
`save_ASCII_seismograms`: Outputs seismograms in two-column ASCII files. Useful for quick plotting

In [ ]:
! mpirun -n 4 bin/xspecfem2D > OUTPUT_FILES/output_solver.txt

In [ ]:
! ls OUTPUT_FILES

!!! Go through the solver log here !!!

### 5) Understanding SPECFEM2D Output Files

The `xspecfem2D` file has created a plethora of results! Let's have a look one by one to see what each of these files are, and how they can help us understand our simulation.

#### a) Velocity Model

`xspecfem2D` outputs the velocity model into the *DATA/* directory. We can use some utility functions written into `SeisFlows` to plot this model to help us visualize our domain.

In [ ]:
# The .bin files define our velocity model
! ls DATA/*bin

In [ ]:
# We can use SeisFlows to plot this model
from seisflows.tools.specfem import Model

m = Model(path="DATA")
m.plot2d(parameter="vs")

From the model figure above, which shows shear wave velocities in a 2D domain, we can see that our model is defined by 3 distinct layers. The top layer, from Z=3500m down to Z=2000m, features a moderate velocity with topography at the surface (Z>3000m). The middle layer, from Z=2000m down to Z=1000m shows a low velocity zone with a high-velocity column (turquoise square). Finally, the bottom layer, from Z=1000m to Z=0m, features a realtively fast velocity. 

Each interface (topography and contact between layers), was defined in a file specified by `Par_file` parameter `interfacesfile`.

In [ ]:
! seisflows sempar -P DATA/Par_file interfacesfile

In [ ]:
! cat EXAMPLES/simple_topography_and_also_a_simple_fluid_layer/DATA/interfaces_simple_topo_curved.dat

### b) Synthetic waveforms

During the simulation, `xspecfem2D` initiated the `SOURCE` file at time 0. Over the course of the simulation, seismic waves propogated outward and were recorded at synthetic receiver locations defined by the `STATIONS` file. Each `STATION` therefore has a corresponding synthetic seismogram located in the *OUTPUT_FILES/* directory.

>__NOTE:__ Because we specified that we wanted displacement seismograms with `Par_file` parameter `seismotype`, our synthetics have the file extension `.semd`, where 'd' stands for displacement. Velocity seismograms would be extension'ed `.semv`.

In [ ]:
! ls OUTPUT_FILES/*.semd

Synthetic waveforms can be generated in a variety of formats. For simplicity we have chosen to output our synthetics in ACSII format. These ASCII files are two columns, representing time and amplitude, respectively.

In [ ]:
# The first 10 lines of a seismogram
! head -10 OUTPUT_FILES/AA.S0001.BXX.semd

In [ ]:
# We can easily plot these using NumPy and Matplotlib
import numpy as np
import matplotlib.pyplot as plt

data = np.loadtxt("OUTPUT_FILES/AA.S0001.BXX.semd", dtype=float)
plt.plot(data[:,0], data[:,1], c="k")
plt.title("AA.S0001.BXX.semd")
plt.xlabel("Time [s]")
plt.ylabel("Displacement [m]")

In [ ]:
# SeisFlows also has a simple command line tool to plot seismograms using ObsPy
! seisflows plotst OUTPUT_FILES/AA.S0001.BXX.semd --savefig AA.S0001.BXX.semd.png

In [ ]:
# We need to use IPython to visualize these figures in the notebook
from IPython.display import Image
Image("AA.S0001.BXX.semd.png")

In [ ]:
# We can use wildcards to plot multiple waveforms at once
! seisflows plotst OUTPUT_FILES/AA.S000?.BXX.semd --savefig AA.S000n.BXX.semd.png
Image("AA.S000n.BXX.semd.png")

### c) SPECFEM2D Figures

`xspecfem2D` also generates snapshots of the forward wavefield. These are automatically generated during a simulation as .jpg files. The `Par_file` parameter `NTSTEP_BETWEEN_OUTPUT_IMAGES` controls how often it generates figures during a simulation. We can see below that `DT`=.0011, so we output images every 0.11s of simulation time. 

In [ ]:
! seisflows sempar -P DATA/Par_file ntstep_between_output_images
! seisflows sempar -P DATA/Par_file dt

In [ ]:
! ls OUTPUT_FILES/*.jpg

In [ ]:
# Forward wavefield at NSTEP=100, T=.11s
Image("OUTPUT_FILES/forward_image000000100.jpg")

In [ ]:
# Forward wavefield at NSTEP=400, T=.44s
Image("OUTPUT_FILES/forward_image000000400.jpg")

In [ ]:
# Forward wavefield at NSTEP=700, T=.77s
Image("OUTPUT_FILES/forward_image000000700.jpg")

## 6) Conclusions

In this notebook we explored SPECFEM2D, and learned to run the default example mesh generation and forward simulation. We took a look at the most important files required for a simulation, and how User's can manipulate various parameters and files to run their own simulations. Finally we had a look at the results of a SPECFEM2D simulation, including waveforms, models, and wavefield snapshots.